## Binance top 200 coins by volume and momentum

In [104]:
coins = 15

## 1- Obtengo la data usando la API de Binance

In [105]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_top_volume_coins_list_api(count=200):
    # Obtener la lista de todos los símbolos de trading
    symbols_url = "https://api.binance.com/api/v3/exchangeInfo"
    symbols_response = requests.get(symbols_url)
    symbols_data = symbols_response.json()
    
    # Filtrar solo los pares con USDT
    usdt_symbols = [symbol['symbol'] for symbol in symbols_data['symbols'] if symbol['symbol'].endswith('USDT')]

    # Elimino las stablecoins
    stablecoins = ['USDCUSDT', 'FDUSDUSDT', '1000SATS', '1000SATSBTC']
    usdt_symbols = [symbol for symbol in usdt_symbols if symbol not in stablecoins]
    
    # Calcular la fecha de inicio (7 días atrás)
    start_time = int((datetime.now() - timedelta(days=7)).timestamp() * 1000)
    
    # Obtener el volumen para cada símbolo
    volumes = []
    for symbol in usdt_symbols:
        klines_url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_time}"
        klines_response = requests.get(klines_url)
        klines_data = klines_response.json()
        
        # Calcular el volumen promedio
        total_volume = sum(float(kline[7]) for kline in klines_data) #7 es quote asset volume
        avg_volume = total_volume / len(klines_data) if klines_data else 0
        volumes.append((symbol[:-4], avg_volume))  # Removemos 'USDT' del símbolo
    
    # Ordenar por volumen descendente y tomar los primeros 'count'
    top_volumes = sorted(volumes, key=lambda x: x[1], reverse=True)[:count]
    
    return top_volumes  # Ahora retornamos la lista completa de tuplas (moneda, volumen)

def get_top_volume_coins_df_api(count=200):
    volumes = get_top_volume_coins_list_api(count)
    df = pd.DataFrame(volumes, columns=['coin', 'average volume last 7 days'])

    # Ordenar el DataFrame antes de formatear los números
    df = df.sort_values('average volume last 7 days', ascending=False)

   # Formatear los números después de ordenar
    df['average volume last 7 days'] = df['average volume last 7 days'].apply(lambda x: f'{x:.2f}')

    return df

In [106]:
api_top_coins_list = get_top_volume_coins_list_api(coins)
#api_top_coins_list

In [107]:
api_top_coins_df = get_top_volume_coins_df_api(coins)
api_top_coins_df

,coin,average volume last 7 days
0,BTC,1447576029.59
1,ETH,670677871.86
2,SOL,357754695.90
3,BNB,109770757.04
4,WIF,109090387.52
5,PEPE,105078333.02
6,DOGS,102575851.96
7,XRP,96501442.83
8,1000SATS,88609725.68
9,SUI,70489296.85


#### Obtengo la data usando el wraper de Python de Binance

In [108]:
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta

# Inicializar el cliente de Binance
# Nota: Para el uso de la API pública, no necesitamos proporcionar claves API
client = Client()

def get_top_volume_coins_list_wraper(count=200):
    # Obtener la información de todos los símbolos de trading
    exchange_info = client.get_exchange_info()
    
    # Filtrar solo los pares con USDT
    usdt_symbols = [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['symbol'].endswith('USDT')]
    
    # Calcular la fecha de inicio (7 días atrás)
    start_time = int((datetime.now() - timedelta(days=7)).timestamp() * 1000)
    
    # Obtener el volumen para cada símbolo
    volumes = []
    for symbol in usdt_symbols:
        klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1DAY, startTime=start_time)
        
        # Calcular el volumen promedio
        total_volume = sum(float(kline[7]) for kline in klines) #7 es quote asset volume
        avg_volume = total_volume / len(klines) if klines else 0
        volumes.append((symbol[:-4], avg_volume))  # Removemos 'USDT' del símbolo
    
    # Ordenar por volumen descendente y tomar los primeros 'count'
    top_volumes = sorted(volumes, key=lambda x: x[1], reverse=True)[:count]
    
    return top_volumes  # Ahora retornamos la lista completa de tuplas (moneda, volumen)

def get_top_volume_coins_df_wraper(count=200):
    volumes = get_top_volume_coins_list_wraper(count)
    df = pd.DataFrame(volumes, columns=['coin', 'average volume last 7 days'])

    # Ordenar el DataFrame antes de formatear los números
    df = df.sort_values('average volume last 7 days', ascending=False)

    # Formatear los números después de ordenar
    df['average volume last 7 days'] = df['average volume last 7 days'].apply(lambda x: f'{x:.2f}')
    

    return df


In [109]:
#wraper_top_coins_list = get_top_volume_coins_list_wraper(coins)
#wraper_top_coins_list

In [110]:
#wraper_top_coins_df = get_top_volume_coins_df_wraper(coins)
#wraper_top_coins_df

#### Conclusion
Mejor uso el API directamente, ya que evito que CS50 tenga que instalar un paquete adicional de python. 

Las variables que pasan a la siguiente etapa son: 


api_top_coins_df 

api_top_coins_list

Trabajare con la lista pero primero tengo que eliminar los valores de volumen, que hago a continuacion

In [111]:
# Get the ticker list for coins.
top_coins = [item[0] for item in api_top_coins_list]
top_coins

['BTC',
 'ETH',
 'SOL',
 'BNB',
 'WIF',
 'PEPE',
 'DOGS',
 'XRP',
 '1000SATS',
 'SUI',
 'EURI',
 'DOGE',
 'AAVE',
 'ORDI',
 'TRX']

## 2- Obtener klines para cada criptomoeda

In [112]:
# Obtiene klines con la lista top_coins, el par BTC y el par USDT
import pandas as pd
from binance.client import Client
from datetime import datetime, timedelta
import os
import sqlite3

client = Client()

not_available_btcpair = []

def get_klines_btcpair(crypto_list, lookback, reference_low):
    """
    loopback: número de días a retroceder desde la fecha de referencia
    La funcion hace algo que no necesito, le paso una fecha reference_low y luego calcula los loopback dias atras antes de esa fecha para jalar la data de binance, hasta el dia de hoy? No es necesario
    """

    # Convertir la fecha de referencia a timestamp
    reference_date = datetime.strptime(reference_low, "%d-%m-%Y")
    start_date = reference_date - timedelta(days=lookback)
    start_timestamp = int(start_date.timestamp() * 1000)

    dataframes = []
    

    # Crear la carpeta 'data' si no existe
    if not os.path.exists('data'):
        os.makedirs('data')

    for crypto in crypto_list:
        # Obtener los datos de Binance

        try:
            print(f"Fetching data for {crypto}USDT...")
            klines = client.get_historical_klines(
                symbol=f"{crypto}BTC", # for BTC pairs
                #symbol=f"{crypto}USDT", 
                interval=Client.KLINE_INTERVAL_1DAY,
                start_str=start_timestamp
            )

            # Crear el DataFrame
            df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            
            # Convertir el timestamp a datetime
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Establecer el timestamp como índice
            df.set_index('timestamp', inplace=True)
            
            # Convertir las columnas necesarias a float
            for col in ['open', 'high', 'low', 'close', 'volume']:
                df[col] = df[col].astype(float)
            
            # Guardar el DataFrame como CSV
            filename = f"data/{crypto}BTC_DF_{lookback}.csv"
            df.to_csv(filename)
            
            dataframes.append(df)
            print(f"Datos de {crypto} guardados en {filename}")

        except:
            print(f"Error al obtener datos para {crypto}.")
            not_available_btcpair.append(crypto)
            continue

    return dataframes

def get_klines_usdtpair(crypto_list, lookback, reference_low):
    # Convertir la fecha de referencia a timestamp
    reference_date = datetime.strptime(reference_low, "%d-%m-%Y")
    start_date = reference_date - timedelta(days=lookback)
    start_timestamp = int(start_date.timestamp() * 1000)

    dataframes = []

    # Crear la carpeta 'data' si no existe
    if not os.path.exists('data'):
        os.makedirs('data')

    for crypto in crypto_list:
        # Obtener los datos de Binance

        try:
            print(f"Fetching data for {crypto}USDT...")
            klines = client.get_historical_klines(
                symbol=f"{crypto}USDT", # for USDT pairs
                interval=Client.KLINE_INTERVAL_1DAY,
                start_str=start_timestamp
            )

            # Crear el DataFrame
            df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            
            # Convertir el timestamp a datetime
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Establecer el timestamp como índice
            df.set_index('timestamp', inplace=True)
            
            # Convertir las columnas necesarias a float
            for col in ['open', 'high', 'low', 'close', 'volume']:
                df[col] = df[col].astype(float)
            
            # Guardar el DataFrame como CSV
            filename = f"data/{crypto}BTC_DF_{lookback}.csv"
            df.to_csv(filename)
            
            dataframes.append(df)
            print(f"Datos de {crypto} guardados en {filename}")
        except:
            print(f"Error al obtener datos para {crypto}.")
            continue

    return dataframes

# Ejemplo de uso:
# crypto_list = ['BTC', 'ETH', 'ADA']
# dataframes = get_crypto_data(crypto_list, 30, "01-06-2023")
# create_db(crypto_list)

dataframes = get_klines_btcpair(top_coins, 600, "05-07-2024")

Fetching data for BTCUSDT...
Error al obtener datos para BTC.
Fetching data for ETHUSDT...
Datos de ETH guardados en data/ETHBTC_DF_600.csv
Fetching data for SOLUSDT...
Datos de SOL guardados en data/SOLBTC_DF_600.csv
Fetching data for BNBUSDT...
Datos de BNB guardados en data/BNBBTC_DF_600.csv
Fetching data for WIFUSDT...
Datos de WIF guardados en data/WIFBTC_DF_600.csv
Fetching data for PEPEUSDT...
Error al obtener datos para PEPE.
Fetching data for DOGSUSDT...
Error al obtener datos para DOGS.
Fetching data for XRPUSDT...
Datos de XRP guardados en data/XRPBTC_DF_600.csv
Fetching data for 1000SATSUSDT...
Error al obtener datos para 1000SATS.
Fetching data for SUIUSDT...
Datos de SUI guardados en data/SUIBTC_DF_600.csv
Fetching data for EURIUSDT...
Error al obtener datos para EURI.
Fetching data for DOGEUSDT...
Datos de DOGE guardados en data/DOGEBTC_DF_600.csv
Fetching data for AAVEUSDT...
Datos de AAVE guardados en data/AAVEBTC_DF_600.csv
Fetching data for ORDIUSDT...
Datos de ORDI 

In [113]:
dataframes[0]

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2022-11-14,0.074803,0.075520,0.073589,0.074792,115945.7535,1668470399999,8659.49382282,331084,60971.55460000,4554.93581501,0
2022-11-15,0.074793,0.075579,0.073670,0.074168,89339.7846,1668556799999,6680.80713730,250952,43927.52840000,3285.62394612,0
2022-11-16,0.074168,0.074857,0.072353,0.072993,89619.4520,1668643199999,6589.76572372,249903,44182.21900000,3248.84447608,0
2022-11-17,0.072993,0.073370,0.071707,0.071913,86185.1891,1668729599999,6244.22208675,234949,39356.31260000,2853.82711938,0
2022-11-18,0.071913,0.072970,0.071595,0.072614,55965.4794,1668815999999,4056.02119218,138972,28913.50580000,2095.54078772,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-05,0.042260,0.042450,0.041810,0.042160,17020.4826,1725580799999,717.97236428,109254,8371.67310000,353.15594611,0
2024-09-06,0.042170,0.042580,0.040850,0.041230,46145.7447,1725667199999,1930.32483583,227897,23096.34040000,966.78701183,0
2024-09-07,0.041240,0.042270,0.041230,0.041990,20385.5488,1725753599999,853.72200994,112353,9323.97520000,390.56535641,0


In [114]:
not_available_btcpair

['BTC', 'PEPE', 'DOGS', '1000SATS', 'EURI']

## Analisis core de las monedas mas importantes de Binance

In [115]:
import pandas as pd
import os
from datetime import datetime

def analyze_momentum(pivot_low_date, reference_high_date):
    # Convertir las fechas de string a datetime, fechas en formato DD-MM-YYYY
    pivot_low_date = datetime.strptime(pivot_low_date, "%d-%m-%Y")
    reference_high_date = datetime.strptime(reference_high_date, "%d-%m-%Y")

    results = []

    # Leer todos los archivos CSV en la carpeta 'data'
    for filename in os.listdir('data'):
        print(filename)

        if filename.endswith('.csv'):
            coin = filename.split('_')[0]  # Extraer el nombre de la moneda del nombre del archivo
            df = pd.read_csv(os.path.join('data', filename), index_col='timestamp', parse_dates=True)

            # Asegurarse de que las columnas necesarias son de tipo float
            for col in ['low', 'high']:
                df[col] = df[col].astype(float)

            # Manejo de excepciones si no se encuentra la fecha
            try:

                # Encontrar el pivot_low
                pivot_low = df.loc[pivot_low_date.strftime("%Y-%m-%d"), 'low']

                # Encontrar el max_high después del pivot_low_date
                max_high = df.loc[pivot_low_date:, 'high'].max()

                # Encontrar el reference_high
                reference_high = df.loc[reference_high_date.strftime("%Y-%m-%d"), 'high']

                # Calcular el cambio porcentual
                change_pct = ((reference_high - pivot_low) / pivot_low) * 100

                results.append({
                    'coin': coin,
                    'pivot_low': pivot_low,
                    'max_high': max_high,
                    'reference_high': reference_high,
                    'change_pct': change_pct
                })
            except KeyError:
                print (f"se ha encontrado un keyerror para {filename}")

    # Crear el DataFrame final
    result_df = pd.DataFrame(results)

    # Ordenar el DataFrame por change_pct de mayor a menor
    result_df = result_df.sort_values('change_pct', ascending=False)

    return result_df

# Ejemplo de uso:
aa_df = analyze_momentum(pivot_low_date="05-08-2024", reference_high_date="25-08-2024")
aa_df

ETC_DF_600.csv
RENDER_DF_600.csv
BTC_DF_600.csv
BNX_DF_600.csv
DATA_DF_600.csv
AR_DF_600.csv
SUN_DF_600.csv
DOGE_DF_600.csv
AEVO_DF_600.csv
AAVEBTC_DF_600.csv
ARKM_DF_600.csv
LOKA_DF_600.csv
DAR_DF_600.csv
ARB_DF_600.csv
DOGS_DF_600.csv
se ha encontrado un keyerror para DOGS_DF_600.csv
SEI_DF_600.csv
PEPE_DF_600.csv
TIA_DF_600.csv
DOT_DF_600.csv
VIDT_DF_600.csv
JTO_DF_600.csv
GMT_DF_600.csv
TON_DF_600.csv
se ha encontrado un keyerror para TON_DF_600.csv
BONK_DF_600.csv
DYDX_DF_600.csv
RDNT_DF_600.csv
FTM_DF_600.csv
MATIC_DF_600.csv
IDEX_DF_600.csv
1INCH_DF_600.csv
JASMY_DF_600.csv
CHESS_DF_600.csv
JUP_DF_600.csv
BAKE_DF_600.csv
TRB_DF_600.csv
OM_DF_600.csv
FLOKI_DF_600.csv
XRPBTC_DF_600.csv
IO_DF_600.csv
PENDLE_DF_600.csv
LDO_DF_600.csv
ORDIBTC_DF_600.csv
SHIB_DF_600.csv
FIL_DF_600.csv
NOT_DF_600.csv
TRXBTC_DF_600.csv
1000SATS_DF_600.csv
SUIBTC_DF_600.csv
USDC_DF_600.csv
LINK_DF_600.csv
FLUX_DF_600.csv
SLF_DF_600.csv
se ha encontrado un keyerror para SLF_DF_600.csv
BCH_DF_600.csv
PIXEL

,coin,pivot_low,max_high,reference_high,change_pct
6,SUN,0.008030,0.044840,0.044840,458.405978
109,RARE,0.056400,0.347600,0.259700,360.460993
18,VIDT,0.013830,0.054790,0.045500,228.994939
106,1000SATSBTC,0.000153,0.000428,0.000427,178.459530
44,1000SATS,0.000153,0.000428,0.000427,178.459530
...,...,...,...,...,...
59,BNBBTC,0.008121,0.009744,0.009072,11.710380
61,DOGEBTC,0.000002,0.000002,0.000002,11.392405
101,EUR,1.078100,1.129900,1.129900,4.804749
108,FDUSD,0.992400,1.202600,1.000500,0.816203


In [116]:
#Valido si realmente los valores que me entregan corresponden a los de binance
# La data es correcta para BTC
# La data es correcta 



In [117]:
# segunda version con fomrato YYYY-MM-DD

import os
from datetime import datetime

def analyze_momentum(pivot_low_date, reference_high_date):
    # Las fechas ya están en formato YYYY-MM-DD, no necesitamos convertirlas

    results = []

    # Leer todos los archivos CSV en la carpeta 'data'
    for filename in os.listdir('data'):
        if filename.endswith('.csv'):
            coin = filename.split('_')[0]  # Extraer el nombre de la moneda del nombre del archivo
            df = pd.read_csv(os.path.join('data', filename), index_col='timestamp', parse_dates=True)

            # Asegurarse de que las columnas necesarias son de tipo float
            for col in ['low', 'high']:
                df[col] = df[col].astype(float)

            # Encontrar el pivot_low
            pivot_low = df.loc[pivot_low_date, 'low']

            # Encontrar el max_high después del pivot_low_date
            max_high = df.loc[pivot_low_date:, 'high'].max()

            # Encontrar el reference_high
            reference_high = df.loc[reference_high_date, 'high']

            # Calcular el cambio porcentual
            change_pct = ((reference_high - pivot_low) / pivot_low) * 100

            results.append({
                'coin': coin,
                'pivot_low': pivot_low,
                'max_high': max_high,
                'reference_high': reference_high,
                'change_pct': change_pct
            })

    # Crear el DataFrame final
    result_df = pd.DataFrame(results)

    # Ordenar el DataFrame por change_pct de mayor a menor
    result_df = result_df.sort_values('change_pct', ascending=False)

    return result_df

# Ejemplo de uso:
#aa_df = analyze_momentum(pivot_low_date="2024-08-5", reference_high_date="2024-08-25")
#aa_df